In [208]:
from __future__ import print_function, division

from sympy import S, Expr, Mul,Add, Pow, symbols, Function
from sympy.physics.quantum import Operator
from sympy.printing.latex import print_latex
from sympy.printing.pretty.stringpict import prettyForm


from sympy import S, Expr, Mul, Add, Pow
from sympy.printing.pretty.stringpict import prettyForm

from sympy.physics.quantum.dagger import Dagger
from sympy.physics.quantum.operator import Operator

from sympy.core.decorators import _sympifyit, call_highest_priority
from sympy.core.function import UndefinedFunction

In [1105]:
class PoissonBracket(Expr):
    """
    Operator class for the Poisson Bracket - allows to have it as a symbolic operator
    """
    
#     _op_priority = 11.0
    _is_commutative = False
    
    def __new__(cls,A,B,**kwargs):       
        r=cls.eval(A,B)
        if r is not None:
            return r
        obj = Expr.__new__(cls,A,B)
        obj.A = A
        obj.B = B
        obj.indep_coords = kwargs.get("coords",None)
        obj.indep_mom = kwargs.get("mom",None)
        return obj
    
    @classmethod
    def eval(cls,a,b):
        if not (a and b):
#             print("here")
            return S.Zero
        if a==b:
            print("here")
            print(a,b)
            return S.Zero
#         try:
#             if a.compare(b) ==1:
#                 return S.NegativeOne*cls(b,a)
#         except TypeError:
#             tmp = symbols("tmp")
#             if a(tmp).compare(b(tmp)) ==1:
#                         return S.NegativeOne*cls(b,a)
                
    @property
    def free_symbols(self):
        return self.A.free_symbols.union(self.B.free_symbols)
    
    @property
    def expr(self):
#         print(self.indep_coords)
        if (self.indep_coords and self.indep_mom):
            hp = S(0)
            for qi, pi in zip(self.indep_coords, self.indep_mom):
                hp += self.A.diff(qi) * self.B.diff(pi)
                hp -= self.A.diff(pi) * self.B.diff(qi)
            return hp
        else:
            return self
        
    def _eval_derivative(self, symbol):
        """
        Necessary to get the full expanded expression
        when one of the arguments is itself a PoissonBracket.
        In other words to allow evalutation of nested Poisson Brackets.
        """
        new_expr = self.expr.diff(symbol)
        return new_expr
    
    def doit(self, **hints):
        """ Evaluate commutator """
        A = self.args[0]
        B = self.args[1]
        
        if isinstance(A, Operator) and isinstance(B, Operator):
            try:
                comm = A._eval_commutator(B, **hints)
            except NotImplementedError:
                try:
                    comm = -1*B._eval_commutator(A, **hints)
                except NotImplementedError:
                    comm = None
            if comm is not None:
                return comm.doit(**hints)
        if isinstance(A, UndefinedFunction) and isinstance(B, UndefinedFunction):
            # add back the coords and momenta and create new poissonbracket with it
            # then appply doit.
            return self
        
        
        hp = self.expr
        return hp
    
    def _eval_expand_commutator(self, **hints):
        A = self.A #self.args[0]
        B = self.B #self.args[1]

        if isinstance(A, Add):
            # [A + B, C]  ->  [A, C] + [B, C]
            sargs = []
            for term in A.args:
                comm = PoissonBracket(term, B)
                print(comm.__class__)
                if isinstance(comm, PoissonBracket):
                    comm = comm._eval_expand_commutator()
                sargs.append(comm)
#             print(sargs)
            return Add(*sargs)
        elif isinstance(B, Add):
            # [A, B + C]  ->  [A, B] + [A, C]
            sargs = []
            for term in B.args:
                comm = PoissonBracket(A, term)
                if isinstance(comm, PoissonBracket):
                    comm = comm._eval_expand_commutator()
                sargs.append(comm)
            return Add(*sargs)
        elif isinstance(A, Mul):
            # [A*B, C] -> A*[B, C] + [A, C]*B
            a = A.args[0]
            b = Mul(*A.args[1:])
            c = B
            comm1 = PoissonBracket(b, c)
            comm2 = PoissonBracket(a, c)
            if isinstance(comm1, PoissonBracket):
                comm1 = comm1._eval_expand_commutator()
            if isinstance(comm2, PoissonBracket):
                comm2 = comm2._eval_expand_commutator()
            first = Mul(a, comm1)
            second = Mul(comm2, b)
#             print(first,second)
            return Add(first, second)
        elif isinstance(B, Mul):
            # [A, B*C] -> [A, B]*C + B*[A, C]
            a = A
            b = B.args[0]
            c = Mul(*B.args[1:])
            comm1 = PoissonBracket(a, b)
            comm2 = PoissonBracket(a, c)
            if isinstance(comm1, PoissonBracket):
                comm1 = comm1._eval_expand_commutator()
            if isinstance(comm2, PoissonBracket):
                comm2 = comm2._eval_expand_commutator()
            first = Mul(comm1, c)
            second = Mul(b, comm2)
            return Add(first, second)
        elif isinstance(A, Pow):
            # [A**n, C] -> A**(n - 1)*[A, C] + A**(n - 2)*[A, C]*A + ... + [A, C]*A**(n-1)
            return self._expand_pow(A, B, 1)
        elif isinstance(B, Pow):
            # [A, C**n] -> C**(n - 1)*[C, A] + C**(n - 2)*[C, A]*C + ... + [C, A]*C**(n-1)
            return self._expand_pow(B, A, -1)
        return self
    
    def _expand_pow(self, A, B, sign):
        exp = A.exp
        if not exp.is_integer or not exp.is_constant() or abs(exp) <= 1:
            # nothing to do
            return self
        base = A.base
        if exp.is_negative:
            base = A.base**-1
            exp = -exp
        comm = PoissonBracket(base, B).expand(commutator=True)

        result = base**(exp - 1) * comm
        for i in range(1, exp):
            result += base**(exp - 1 - i) * comm * base**i
        return sign*result.expand()
    
    
    
    def _latex(self, printer, *args):
#         print(tuple([
#             printer._print(arg.func, *args) if arg.__class__.__name__ != "PoissonBracket" else  printer._print(arg, *args) for arg in self.args[:2] ]))
        news = []
        for arg in self.args[:2]:
#             print(type(arg))
            if isinstance(arg,Add):
                news.append(" + ".join([printer.doprint(a.func, *args) if a.is_Function  else  printer.doprint(a, *args) for a in arg.args]))
            
            elif isinstance(arg,PoissonBracket):
                news.append(printer.doprint(arg,*args))
#                 news.append("\\lbrace %s,%s\\rbrace" %tuple([
#             printer.doprint(a.func, *args) if a.is_Function and not isinstance(a,UndefinedFunction)  else  printer.doprint(a, *args) for a in arg.args[:2] ]))
            
            elif isinstance(arg,Mul):
                 news.append(" * ".join([printer.doprint(a.func, *args) if a.is_Function  else  printer.doprint(a, *args) for a in arg.args]))
#                 print([a for a in arg.args])
#                 news.append(" * ".join([ 
#                     printer._print(a.func, *args) 
#                     if a.is_Function  and not (isinstance(a,UndefinedFunction))
#                     else  printer._print(a, *args) for a in args
#                 ]))
            elif isinstance(arg,Pow):
                if isinstance(arg.args[0], UndefinedFunction):
                    news.append("{}^{}".format([printer.doprint(a) for a in arg.args]))
                elif isinstance(arg.args[0], Function):
                    news.append("{}^{}".format(printer.doprint(arg.args[0].func,*args), printer.doprint(arg.args[1],*args)))
                else:
                    news.append(printer.doprint(arg, *args))
                
            elif isinstance(arg, Function) and not (isinstance(arg,UndefinedFunction)):
                news.append(printer.doprint(arg.func, *args))
                                                    
            else: # isinstance(arg, Function) and isinstance(arg,UndefinedFunction):
                news.append(printer.doprint(arg, *args))
    
        print(news)
        return "\\lbrace %s,%s\\rbrace" %tuple(news)
        
        return "\\lbrace %s,%s\\rbrace" %tuple([
            printer._print(arg.func, *args) if arg.is_Function  and not (isinstance(arg,UndefinedFunction))
            else  printer._print(arg, *args) 
            for arg in self.args[:2] ])
#         return "\\lbrace %s,%s\\rbrace" % tuple([
#             printer._print(arg, *args) for arg in self.args[:2]])

In [1106]:
1

x, px, y, py, z, delta, eps, h , k0, k1, k2, sigma, betag ,f = symbols('x p_x y p_y z delta epsilon h k_0 k_1 k_2 sigma beta_gamma f', real=True)

2

L, beta0, gamma0 = symbols(" L beta_0 gamma_0", real=True, positive=True)

In [1107]:
isinstance(A,UndefinedFunction)

True

In [1108]:
A = Function("A", commutative=False)
B = Function("B", commutative=False)
C = Function("C", commutative=False)
D = Function("D", commutative=False)

In [1109]:
test1 =PoissonBracket(A,B)
test1

['A', 'B']


PoissonBracket(A, B)

In [1110]:
test1 == PoissonBracket(A,B)

True

In [1111]:
test1.doit()

['A', 'B']


PoissonBracket(A, B)

In [1112]:
test1.A= A(x)

In [1113]:
test1.free_symbols

{x}

In [1114]:
test2 = PoissonBracket(x**2,x+px)
test2.doit()

['x^{2}', 'p_{x} + x']


PoissonBracket(x**2, p_x + x)

In [1115]:
[i for i in test2.free_symbols][0],0

(p_x, 0)

In [1116]:
test3 = PoissonBracket(A(x,px),B(y,py))
test3

['A', 'B']


PoissonBracket(A(x, p_x), B(y, p_y))

In [1117]:
test3.doit()

['A', 'B']


PoissonBracket(A(x, p_x), B(y, p_y))

In [1118]:
test4 = PoissonBracket(x**2,x+px,coords=[x],mom=[px])
test4

['x^{2}', 'p_{x} + x']


PoissonBracket(x**2, p_x + x)

In [1119]:
test4.doit()

2*x

In [1120]:
test5 = PoissonBracket(A(x,px),B(x,y,py),coords=[x,y],mom=[px,py])
test5

['A', 'B']


PoissonBracket(A(x, p_x), B(x, y, p_y))

In [1121]:
test5.expr,

(-Derivative(A(x, p_x), p_x)*Derivative(B(x, y, p_y), x),)

In [1122]:
test6 = PoissonBracket(test1,C)
test6

['A', 'B']
['\\lbrace A,B\\rbrace', 'C']


PoissonBracket(PoissonBracket(A, B), C)

In [1123]:
test6.doit()

['A', 'B']
['\\lbrace A,B\\rbrace', 'C']


PoissonBracket(PoissonBracket(A, B), C)

In [1124]:
test7 = PoissonBracket(test2,C)
test7

['x^{2}', 'p_{x} + x']
['\\lbrace x^{2},p_{x} + x\\rbrace', 'C']


PoissonBracket(PoissonBracket(x**2, p_x + x), C)

In [1125]:
test7.doit()

['x^{2}', 'p_{x} + x']
['\\lbrace x^{2},p_{x} + x\\rbrace', 'C']


PoissonBracket(PoissonBracket(x**2, p_x + x), C)

In [1126]:
test8 = PoissonBracket(test3,C)
test8

['A', 'B']
['\\lbrace A,B\\rbrace', 'C']


PoissonBracket(PoissonBracket(A(x, p_x), B(y, p_y)), C)

In [1127]:
test9 = PoissonBracket(test4,C)
test9

['x^{2}', 'p_{x} + x']
['\\lbrace x^{2},p_{x} + x\\rbrace', 'C']


PoissonBracket(PoissonBracket(x**2, p_x + x), C)

In [1128]:
test9.doit()

['x^{2}', 'p_{x} + x']
['\\lbrace x^{2},p_{x} + x\\rbrace', 'C']


PoissonBracket(PoissonBracket(x**2, p_x + x), C)

In [1129]:
test10 = PoissonBracket(test5,C)
test10

['A', 'B']
['\\lbrace A,B\\rbrace', 'C']


PoissonBracket(PoissonBracket(A(x, p_x), B(x, y, p_y)), C)

In [1130]:
test10.free_symbols

{p_x, p_y, x, y}

In [1131]:
test11 = PoissonBracket(test5,C(x,px))
test11

['A', 'B']
['\\lbrace A,B\\rbrace', 'C']


PoissonBracket(PoissonBracket(A(x, p_x), B(x, y, p_y)), C(x, p_x))

In [1132]:
test11.doit()

['A', 'B']
['\\lbrace A,B\\rbrace', 'C']


PoissonBracket(PoissonBracket(A(x, p_x), B(x, y, p_y)), C(x, p_x))

In [1133]:
test12 = PoissonBracket(test5,C(x,px),coords=[x,y],mom=[px,py])
test12

['A', 'B']
['\\lbrace A,B\\rbrace', 'C']


PoissonBracket(PoissonBracket(A(x, p_x), B(x, y, p_y)), C(x, p_x))

In [1134]:
test12.doit()

(-Derivative(A(x, p_x), p_x)*Derivative(B(x, y, p_y), (x, 2)) - Derivative(A(x, p_x), p_x, x)*Derivative(B(x, y, p_y), x))*Derivative(C(x, p_x), p_x) + Derivative(A(x, p_x), (p_x, 2))*Derivative(B(x, y, p_y), x)*Derivative(C(x, p_x), x)

In [1135]:
test13 = PoissonBracket(A(x,px)+B(x,y,py),C(x,px),coords=[x,y],mom=[px,py])
test13

['A + B', 'C']


PoissonBracket(A(x, p_x) + B(x, y, p_y), C(x, p_x))

In [1136]:
test13.doit(),

((Derivative(A(x, p_x), x) + Derivative(B(x, y, p_y), x))*Derivative(C(x, p_x), p_x) - Derivative(A(x, p_x), p_x)*Derivative(C(x, p_x), x),)

In [1137]:
test14 = PoissonBracket(A(x,px),B(x,y,py) + C(x,px),coords=[x,y],mom=[px,py])
test14

['A', 'B + C']


PoissonBracket(A(x, p_x), B(x, y, p_y) + C(x, p_x))

In [1138]:
test14.doit(),

(-Derivative(A(x, p_x), p_x)*(Derivative(B(x, y, p_y), x) + Derivative(C(x, p_x), x)) + Derivative(A(x, p_x), x)*Derivative(C(x, p_x), p_x),)

In [1139]:
test15 = PoissonBracket(A(x,px)*B(x,y,py),C(x,px),coords=[x,y],mom=[px,py])
test15

['A * B', 'C']


PoissonBracket(A(x, p_x)*B(x, y, p_y), C(x, p_x))

In [1140]:
test15.doit(),

((A(x, p_x)*Derivative(B(x, y, p_y), x) + Derivative(A(x, p_x), x)*B(x, y, p_y))*Derivative(C(x, p_x), p_x) - Derivative(A(x, p_x), p_x)*B(x, y, p_y)*Derivative(C(x, p_x), x),)

In [1141]:
test16 = PoissonBracket(A(x,px),B(x,y,py)*C(x,px),coords=[x,y],mom=[px,py])
test16

['A', 'B * C']


PoissonBracket(A(x, p_x), B(x, y, p_y)*C(x, p_x))

In [1142]:
test16.doit(),

(-Derivative(A(x, p_x), p_x)*(B(x, y, p_y)*Derivative(C(x, p_x), x) + Derivative(B(x, y, p_y), x)*C(x, p_x)) + Derivative(A(x, p_x), x)*B(x, y, p_y)*Derivative(C(x, p_x), p_x),)

In [1143]:
test17 = PoissonBracket(A(x,px)**3,B(x,y,py),coords=[x,y],mom=[px,py])
test17

['A^3', 'B']


PoissonBracket(A(x, p_x)**3, B(x, y, p_y))

In [1144]:
test17.doit(),

(-3*A(x, p_x)**3*Derivative(A(x, p_x), p_x)*A(x, p_x)**(-1)*Derivative(B(x, y, p_y), x),)

In [1145]:
test18 = PoissonBracket(test12,D(x,y),coords=[x,y],mom=[px,py])
test18

['A', 'B']
['\\lbrace A,B\\rbrace', 'C']
['\\lbrace \\lbrace A,B\\rbrace,C\\rbrace', 'D']


PoissonBracket(PoissonBracket(PoissonBracket(A(x, p_x), B(x, y, p_y)), C(x, p_x)), D(x, y))

In [1152]:
test18.doit() == -((-Derivative(A(x, px), px)*Derivative(B(x, y, py), py, (x, 2)) - Derivative(A(x, px), px, x)*Derivative(B(x, y, py), py, x))*Derivative(C(x, px), px) + Derivative(A(x, px), (px, 2))*Derivative(B(x, y, py), py, x)*Derivative(C(x, px), x))*Derivative(D(x, y), y) - ((-Derivative(A(x, px), px)*Derivative(B(x, y, py), (x, 2)) - Derivative(A(x, px), px, x)*Derivative(B(x, y, py), x))*Derivative(C(x, px), (px, 2)) + (-Derivative(A(x, px), (px, 2))*Derivative(B(x, y, py), (x, 2)) - Derivative(A(x, px), (px, 2), x)*Derivative(B(x, y, py), x))*Derivative(C(x, px), px) + Derivative(A(x, px), (px, 2))*Derivative(B(x, y, py), x)*Derivative(C(x, px), px, x) + Derivative(A(x, px), (px, 3))*Derivative(B(x, y, py), x)*Derivative(C(x, px), x))*Derivative(D(x, y), x)

False

In [1153]:
test18.doit()

-((-Derivative(A(x, p_x), p_x)*Derivative(B(x, y, p_y), p_y, (x, 2)) - Derivative(A(x, p_x), p_x, x)*Derivative(B(x, y, p_y), p_y, x))*Derivative(C(x, p_x), p_x) + Derivative(A(x, p_x), (p_x, 2))*Derivative(B(x, y, p_y), p_y, x)*Derivative(C(x, p_x), x))*Derivative(D(x, y), y) - ((-Derivative(A(x, p_x), p_x)*Derivative(B(x, y, p_y), (x, 2)) - Derivative(A(x, p_x), p_x, x)*Derivative(B(x, y, p_y), x))*Derivative(C(x, p_x), (p_x, 2)) + (-Derivative(A(x, p_x), (p_x, 2))*Derivative(B(x, y, p_y), (x, 2)) - Derivative(A(x, p_x), (p_x, 2), x)*Derivative(B(x, y, p_y), x))*Derivative(C(x, p_x), p_x) + Derivative(A(x, p_x), (p_x, 2))*Derivative(B(x, y, p_y), x)*Derivative(C(x, p_x), p_x, x) + Derivative(A(x, p_x), (p_x, 3))*Derivative(B(x, y, p_y), x)*Derivative(C(x, p_x), x))*Derivative(D(x, y), x)

In [1156]:
test18.doit(),

(-((-Derivative(A(x, p_x), p_x)*Derivative(B(x, y, p_y), p_y, (x, 2)) - Derivative(A(x, p_x), p_x, x)*Derivative(B(x, y, p_y), p_y, x))*Derivative(C(x, p_x), p_x) + Derivative(A(x, p_x), (p_x, 2))*Derivative(B(x, y, p_y), p_y, x)*Derivative(C(x, p_x), x))*Derivative(D(x, y), y) - ((-Derivative(A(x, p_x), p_x)*Derivative(B(x, y, p_y), (x, 2)) - Derivative(A(x, p_x), p_x, x)*Derivative(B(x, y, p_y), x))*Derivative(C(x, p_x), (p_x, 2)) + (-Derivative(A(x, p_x), (p_x, 2))*Derivative(B(x, y, p_y), (x, 2)) - Derivative(A(x, p_x), (p_x, 2), x)*Derivative(B(x, y, p_y), x))*Derivative(C(x, p_x), p_x) + Derivative(A(x, p_x), (p_x, 2))*Derivative(B(x, y, p_y), x)*Derivative(C(x, p_x), p_x, x) + Derivative(A(x, p_x), (p_x, 3))*Derivative(B(x, y, p_y), x)*Derivative(C(x, p_x), x))*Derivative(D(x, y), x),)

In [1147]:
test19 = PoissonBracket(test13,D(x,y),coords=[x,y],mom=[px,py])
test19

['A + B', 'C']
['\\lbrace A + B,C\\rbrace', 'D']


PoissonBracket(PoissonBracket(A(x, p_x) + B(x, y, p_y), C(x, p_x)), D(x, y))

In [1149]:
test19.doit(),

(-((Derivative(A(x, p_x), x) + Derivative(B(x, y, p_y), x))*Derivative(C(x, p_x), (p_x, 2)) - Derivative(A(x, p_x), p_x)*Derivative(C(x, p_x), p_x, x) - Derivative(A(x, p_x), (p_x, 2))*Derivative(C(x, p_x), x) + Derivative(A(x, p_x), p_x, x)*Derivative(C(x, p_x), p_x))*Derivative(D(x, y), x) - Derivative(B(x, y, p_y), p_y, x)*Derivative(C(x, p_x), p_x)*Derivative(D(x, y), y),)

In [1085]:
test1.indep_coords,test2.indep_coords,test3.indep_coords, test4.indep_coords, test5.indep_coords

(None, None, None, [x], [x, y])

In [505]:
test6.indep_coords,test7.indep_coords,test8.indep_coords, test9.indep_coords, test10.indep_coords, test11.indep_coords

(None, None, None, None, None, None)

In [506]:
test1.indep_mom, test2.indep_mom, test3.indep_mom, test4.indep_mom, test5.indep_mom

(None, None, None, [p_x], [p_x, p_y])

In [507]:
test1.free_symbols, test2.free_symbols, test3.free_symbols, test4.free_symbols, test5.free_symbols

(set(), {p_x, x}, {p_x, p_y, x, y}, {p_x, x}, {p_x, p_y, x, y})

In [508]:
test6.free_symbols, test7.free_symbols, test8.free_symbols, test9.free_symbols, test10.free_symbols, test11.free_symbols

(set(),
 {p_x, x},
 {p_x, p_y, x, y},
 {p_x, x},
 {p_x, p_y, x, y},
 {p_x, p_y, x, y})

In [509]:
test1.expr, test2.expr, test3.expr, test4.expr, test5.expr

(PoissonBracket(A, B),
 PoissonBracket(x**2, p_x + x),
 PoissonBracket(A(x, p_x), B(y, p_y)),
 2*x,
 -Derivative(A(x, p_x), p_x)*Derivative(B(x, y, p_y), x))

In [510]:
test6.expr, test7.expr, test8.expr, test9.expr, test10.expr, test11.expr, test12.expr

(PoissonBracket(PoissonBracket(A, B), C),
 PoissonBracket(PoissonBracket(x**2, p_x + x), C),
 PoissonBracket(PoissonBracket(A(x, p_x), B(y, p_y)), C),
 PoissonBracket(PoissonBracket(x**2, p_x + x), C),
 PoissonBracket(PoissonBracket(A(x, p_x), B(x, y, p_y)), C),
 PoissonBracket(PoissonBracket(A(x, p_x), B(x, y, p_y)), C(x, p_x)),
 (-Derivative(A(x, p_x), p_x)*Derivative(B(x, y, p_y), (x, 2)) - Derivative(A(x, p_x), p_x, x)*Derivative(B(x, y, p_y), x))*Derivative(C(x, p_x), p_x) + Derivative(A(x, p_x), (p_x, 2))*Derivative(B(x, y, p_y), x)*Derivative(C(x, p_x), x))

In [511]:
test1.doit(), test2.doit(), test3.doit(), test4.doit(), test5.doit()

(PoissonBracket(A, B),
 PoissonBracket(x**2, p_x + x),
 PoissonBracket(A(x, p_x), B(y, p_y)),
 2*x,
 -Derivative(A(x, p_x), p_x)*Derivative(B(x, y, p_y), x))

In [512]:
test6.doit(), test7.doit(), test8.doit(), test9.doit(), test10.doit(), test11.doit(), test12.doit()

(PoissonBracket(PoissonBracket(A, B), C),
 PoissonBracket(PoissonBracket(x**2, p_x + x), C),
 PoissonBracket(PoissonBracket(A(x, p_x), B(y, p_y)), C),
 PoissonBracket(PoissonBracket(x**2, p_x + x), C),
 PoissonBracket(PoissonBracket(A(x, p_x), B(x, y, p_y)), C),
 PoissonBracket(PoissonBracket(A(x, p_x), B(x, y, p_y)), C(x, p_x)),
 (-Derivative(A(x, p_x), p_x)*Derivative(B(x, y, p_y), (x, 2)) - Derivative(A(x, p_x), p_x, x)*Derivative(B(x, y, p_y), x))*Derivative(C(x, p_x), p_x) + Derivative(A(x, p_x), (p_x, 2))*Derivative(B(x, y, p_y), x)*Derivative(C(x, p_x), x))

In [513]:
test4.doit()

2*x

In [514]:
test5.doit()

-Derivative(A(x, p_x), p_x)*Derivative(B(x, y, p_y), x)

In [515]:
test12.doit()

(-Derivative(A(x, p_x), p_x)*Derivative(B(x, y, p_y), (x, 2)) - Derivative(A(x, p_x), p_x, x)*Derivative(B(x, y, p_y), x))*Derivative(C(x, p_x), p_x) + Derivative(A(x, p_x), (p_x, 2))*Derivative(B(x, y, p_y), x)*Derivative(C(x, p_x), x)

In [516]:
test13.doit()

(Derivative(A(x, p_x), x) + Derivative(B(x, y, p_y), x))*Derivative(C(x, p_x), p_x) - Derivative(A(x, p_x), p_x)*Derivative(C(x, p_x), x)

In [517]:
test13.expand(commutator=True)

<class '__main__.PoissonBracket'>
<class '__main__.PoissonBracket'>
[PoissonBracket(A(x, p_x), C(x, p_x)), PoissonBracket(B(x, y, p_y), C(x, p_x))]


PoissonBracket(A(x, p_x), C(x, p_x)) + PoissonBracket(B(x, y, p_y), C(x, p_x))

In [518]:
test14.expand(commutator=True)

PoissonBracket(A(x, p_x), B(x, y, p_y)) + PoissonBracket(A(x, p_x), C(x, p_x))

In [519]:
test15.expand(commutator=True)

A(x, p_x)*PoissonBracket(B(x, y, p_y), C(x, p_x)) + PoissonBracket(A(x, p_x), C(x, p_x))*B(x, y, p_y)

In [520]:
test16.expand(commutator=True).doit()

B(x, y, p_y)*PoissonBracket(A(x, p_x), C(x, p_x)) + PoissonBracket(A(x, p_x), B(x, y, p_y))*C(x, p_x)

In [521]:
test17.expand(commutator=True).doit()

A(x, p_x)*PoissonBracket(A(x, p_x), B(x, y, p_y))*A(x, p_x) + A(x, p_x)**2*PoissonBracket(A(x, p_x), B(x, y, p_y)) + PoissonBracket(A(x, p_x), B(x, y, p_y))*A(x, p_x)**2

In [522]:
test18.doit()

-((-Derivative(A(x, p_x), p_x)*Derivative(B(x, y, p_y), p_y, (x, 2)) - Derivative(A(x, p_x), p_x, x)*Derivative(B(x, y, p_y), p_y, x))*Derivative(C(x, p_x), p_x) + Derivative(A(x, p_x), (p_x, 2))*Derivative(B(x, y, p_y), p_y, x)*Derivative(C(x, p_x), x))*Derivative(D(x, y), y) - ((-Derivative(A(x, p_x), p_x)*Derivative(B(x, y, p_y), (x, 2)) - Derivative(A(x, p_x), p_x, x)*Derivative(B(x, y, p_y), x))*Derivative(C(x, p_x), (p_x, 2)) + (-Derivative(A(x, p_x), (p_x, 2))*Derivative(B(x, y, p_y), (x, 2)) - Derivative(A(x, p_x), (p_x, 2), x)*Derivative(B(x, y, p_y), x))*Derivative(C(x, p_x), p_x) + Derivative(A(x, p_x), (p_x, 2))*Derivative(B(x, y, p_y), x)*Derivative(C(x, p_x), p_x, x) + Derivative(A(x, p_x), (p_x, 3))*Derivative(B(x, y, p_y), x)*Derivative(C(x, p_x), x))*Derivative(D(x, y), x)

In [523]:
test18.expand(commutator=True)

PoissonBracket(PoissonBracket(PoissonBracket(A(x, p_x), B(x, y, p_y)), C(x, p_x)), D(x, y))

In [524]:
test19.doit()

-((Derivative(A(x, p_x), x) + Derivative(B(x, y, p_y), x))*Derivative(C(x, p_x), (p_x, 2)) - Derivative(A(x, p_x), p_x)*Derivative(C(x, p_x), p_x, x) - Derivative(A(x, p_x), (p_x, 2))*Derivative(C(x, p_x), x) + Derivative(A(x, p_x), p_x, x)*Derivative(C(x, p_x), p_x))*Derivative(D(x, y), x) - Derivative(B(x, y, p_y), p_y, x)*Derivative(C(x, p_x), p_x)*Derivative(D(x, y), y)

In [525]:
test19.expand(commutator=True)

<class '__main__.PoissonBracket'>
<class '__main__.PoissonBracket'>
[PoissonBracket(A(x, p_x), C(x, p_x)), PoissonBracket(B(x, y, p_y), C(x, p_x))]
<class '__main__.PoissonBracket'>
<class '__main__.PoissonBracket'>
[PoissonBracket(PoissonBracket(A(x, p_x), C(x, p_x)), D(x, y)), PoissonBracket(PoissonBracket(B(x, y, p_y), C(x, p_x)), D(x, y))]


PoissonBracket(PoissonBracket(A(x, p_x), C(x, p_x)), D(x, y)) + PoissonBracket(PoissonBracket(B(x, y, p_y), C(x, p_x)), D(x, y))

In [546]:
from sympy.printing.latex import print_latex
from sympy import latex, Derivative

In [531]:
test19.doit().__class__

sympy.core.add.Add

In [541]:
latex(test19)

'\\lbrace \\lbrace A{\\left(x,p_{x} \\right)} + B{\\left(x,y,p_{y} \\right)},C\\rbrace,D\\rbrace'

In [543]:
print_latex(test19)

\lbrace \lbrace A{\left(x,p_{x} \right)} + B{\left(x,y,p_{y} \right)},C\rbrace,D\rbrace


In [551]:
res12 = (-Derivative(A(x, px), px)*Derivative(B(x, y, py), (x, 2)) - Derivative(A(x, px), px, x)*Derivative(B(x, y, py), x))*Derivative(C(x, px), px) + Derivative(A(x, px), (px, 2))*Derivative(B(x, y, py), x)*Derivative(C(x, px), x)

In [552]:
test12.doit() == res12

True